In [1]:
import matplotlib.pyplot as plt
import scipy.io
import numpy as np
import csv
import pandas as pd
import os,sys

In [2]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *

In [3]:
path_main=Path('/home/visionlab/Georgin/algonautsChallenge_codes/')
# set1
path_training_images_set1=Path(path_main/'Training_Data/92_Image_Set/92images/')
path_training_imagepairs_set1=Path(path_main/'Training_Data/92_Image_Set/92images_imgpairs/')
#set1 testing 
#path_testing_images_set1=Path(path_main/'Training_Data/92_Image_Set/test_92images/')

In [4]:
# READING THE MEG DATA FILE 
# SET1
mat = scipy.io.loadmat(path_training_images_set1/'../target_meg.mat');
meg_rdm_early=mat['MEG_RDMs_early']
meg_rdm_late=mat['MEG_RDMs_late']
# early RDM
meg_rdm_early_timewise=np.mean(meg_rdm_early,axis=0)
mean_early_rdm=np.mean(meg_rdm_early_timewise,axis=0)
# late RDM
meg_rdm_late_timewise=np.mean(meg_rdm_late,axis=0)
mean_late_rdm=np.mean(meg_rdm_late_timewise,axis=0)


In [5]:
# READING FMRI DATA
mat = scipy.io.loadmat(path_training_images_set1/'../target_fmri.mat');
# EVC RDM
fmri_evc_rdm=mat['EVC_RDMs']
mean_fmri_evc_rdm=np.mean(fmri_evc_rdm,axis=0)
# IT RDM
fmri_it_rdm=mat['IT_RDMs']
mean_fmri_it_rdm=np.mean(fmri_it_rdm,axis=0)

In [6]:
mean_fmri_evc_rdm.shape

(92, 92)

In [7]:
# To train using the predefined fast.ai databunch, I are combining the image pair into a single image and storing it in a new folder
# Getting the details about the file in the training  folder
image_file_names=os.listdir(path_training_images_set1)
image_file_names=np.sort(image_file_names)
image_file_names=image_file_names.tolist()

# Storing the combined image and distance
import csv
# creating  a training csv file image1_path, image2_path, similarity measure
from itertools import combinations

N=len(image_file_names)
training_cond=[]
combA = combinations(range(0,N),2)
combA_list=list(combA)
combA=np.array(combA_list)
NA=len(combA)
combB=np.fliplr(combA)

combAB=np.concatenate((combA,combB),axis=0)

# same pairs
#nR=len(combAB)/N
#nR=np.round(nR)
combC=np.array([[i,i]for i in range(0,N)])
#combC=np.repeat(combC,nR,axis=0)

combABC=np.concatenate((combAB,combC),axis=0)

In [8]:
for i in combABC:
    single_condition=['','','','','','']
    if(i[0]!=i[1]):
        value_early=mean_early_rdm[i[0]][i[1]]
        value_late=mean_late_rdm[i[0]][i[1]]
        fmri_value_evc=mean_fmri_evc_rdm[i[0]][i[1]]
        fmri_value_it=mean_fmri_it_rdm[i[0]][i[1]]
    else:
        value_early=0
        value_late=0
        fmri_value_evc=0
        fmri_value_it=0
    
    image1_path=image_file_names[i[0]]
    image2_path=image_file_names[i[1]]
    single_condition[0]=image1_path
    single_condition[1]=image2_path
    single_condition[2]=str(value_early)
    single_condition[3]=str(value_late)
    single_condition[4]=str(fmri_value_evc)
    single_condition[5]=str(fmri_value_it)

    training_cond.append(single_condition) 

from PIL import Image
# Combine  two image and save in new folder

for l in training_cond:
    image1 = Image.open(path_training_images_set1/l[0])
    image2= Image.open(path_training_images_set1/l[1])
    img=Image.new('RGB',(350,175))
    img.paste(image1,(0,0))
    img.paste(image2,(175,0))
    name='set92_'+l[0].strip('.jpg')+'_'+l[1]
    img.save(path_training_imagepairs_set1/name)

In [9]:
with open(path_training_images_set1/'../set1_training_distance_file.csv', 'w',newline='') as csvFile:
    header = ['ImagePair', 'earlyDistance','lateDistance','fmri_evc','fmri_it']
    writer = csv.writer(csvFile,delimiter=',')
    writer.writerow(header)
    for l in training_cond:
        R=['','','','','']
        name='set92_'+l[0].strip('.jpg')+'_'+l[1]
        R[0]=name
        R[1]=l[2]
        R[2]=l[3]
        R[3]=l[4]
        R[4]=l[5]
        writer.writerow(R)
csvFile.close() 

In [10]:
# copying data
import shutil
path_training_image_pairs=Path(path_main/'Training_Data/210_imagepairs/')
for l in training_cond:
    name='set92_'+l[0].strip('.jpg')+'_'+l[1]
    newPath = shutil.copy(path_training_imagepairs_set1/name, path_training_image_pairs)